In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from data_utils import load_images

In [ ]:
from data_utils import ImageDataset


train_dir = "train"  # or the full path if needed

# We load the image_features as a custom dataset class
# The features are the color histograms
train_image_dataset: ImageDataset = load_images(train_dir)
validation_image_dataset: ImageDataset = load_images("validation")
test_image_dataset = load_images("test")
print(f"Loaded {len(train_image_dataset)} images from {train_dir} directory.")
print(f"Loaded {len(validation_image_dataset)} images from validation directory.")
print(f"Loaded {len(test_image_dataset)} images from test directory.")

In [ ]:
from sklearn import svm

print(train_image_dataset.images.shape)
# We define the model as SVC
model = svm.SVC()
# We train the model
model.fit(train_image_dataset.images, train_image_dataset.labels)

In [ ]:
for C in [0.01, 0.1, 1, 10, 100]:
    model = svm.SVC(C=C)
    model.fit(train_image_dataset.images, train_image_dataset.labels)
    score = model.score(
        validation_image_dataset.images, validation_image_dataset.labels
    )
    print(f"C={C}, Validation Score: {score}")

In [ ]:
predictions = model.predict(test_image_dataset.images)
print(f"Predictions for {predictions} images in test dataset.")

In [ ]:
from data_utils import save_predictions_to_csv


save_predictions_to_csv(predictions, test_image_dataset.ids, "predictions.csv")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

val_predictions = model.predict(validation_image_dataset.images)

In [ ]:
cm = confusion_matrix(validation_image_dataset.labels, val_predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix for SVM Model")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

C_values = [0.01, 0.1, 1, 10, 100]
validation_scores = [0.5352, 0.624, 0.7296, 0.7136, 0.7152]

plt.figure(figsize=(8, 5))
plt.plot(C_values, validation_scores, marker="o")
plt.xscale("log")
plt.xlabel("C value")
plt.ylabel("Validation Score")
plt.title("Fine tuning")
plt.grid(True)
plt.show()

In [ ]:
import skops.io as sio

obj = sio.dump(model, "model_svm2.skops")

In [ ]:
import skops.io as sio

model = sio.load("model_svm2.skops")